In [1]:
!pip install -q pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 7.4 MB/s eta 0:00:00


In [2]:
!pip install -q openai pinecone langchain-community tiktoken python-dotenv langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [3]:
import os
import openai
import pinecone
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [ ]:
import os
from google.colab import userdata

# This will prompt you to enter your API keys
os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API key: ").strip()
os.environ["PINECONE_API_KEY"] = input("Enter your Pinecone API key: ").strip()

# Verify the keys are set
print("\nAPI Keys have been set successfully!")
print("OPENAI_API_KEY:", "***" + (os.getenv("OPENAI_API_KEY")[-4:] if os.getenv("OPENAI_API_KEY") else "not set"))
print("PINECONE_API_KEY:", "***" + (os.getenv("PINECONE_API_KEY")[-4:] if os.getenv("PINECONE_API_KEY") else "not set"))

In [10]:
from pinecone import Pinecone, ServerlessSpec

# Initialize the Pinecone client
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create Pinecone index name
index_name = "qa-bot-index"

# Check if index exists, create if not
existing_indexes = [index.name for index in pc.list_indexes()]
print("Existing indexes:", existing_indexes)

if index_name not in existing_indexes:
    print(f"Creating index: {index_name}")
    try:
        # Try with gcp-starter region first (commonly available in free tier)
        pc.create_index(
            name=index_name,
            dimension=1536,  # OpenAI embedding dimension
            metric='cosine',
            spec=ServerlessSpec(
                cloud='gcp',  # Changed to GCP
                region='us-central1'  # GCP region that works with free tier
            )
        )
        print(f"Index {index_name} created successfully in GCP us-central1!")
    except Exception as e:
        print(f"Error with GCP: {str(e)}")
        print("Trying AWS us-east-1...")
        try:
            # If GCP fails, try AWS us-east-1
            pc.create_index(
                name=index_name,
                dimension=1536,
                metric='cosine',
                spec=ServerlessSpec(
                    cloud='aws',
                    region='us-east-1'  # AWS region that works with free tier
                )
            )
            print(f"Index {index_name} created successfully in AWS us-east-1!")
        except Exception as e2:
            print(f"Error with AWS: {str(e2)}")
            print("\nPlease check your Pinecone console for available regions in your plan.")
            raise

# Get the index
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")

Existing indexes: []
Creating index: qa-bot-index
Error with GCP: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '27875f72ed1e7128ed8e5864b41379bb', 'date': 'Fri, 04 Jul 2025 16:43:54 GMT', 'server': 'Google Frontend', 'Content-Length': '202', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Bad request: Your free plan does not support indexes in the us-central1 region of gcp. To create indexes in this region, upgrade your plan."},"status":400}

Trying AWS us-east-1...
Index qa-bot-index created successfully in AWS us-east-1!
Connected to index: qa-bot-index


In [11]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import os

def load_documents(file_path):
    """Load and split documents into chunks."""
    # Load document
    loader = TextLoader(file_path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    return text_splitter.split_documents(documents)

def create_vector_store(documents, index):
    """Create and return a vector store from documents."""
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

    # Create and store vectors in Pinecone
    vector_store = PineconeVectorStore.from_documents(
        documents=documents,
        embedding=embeddings,
        index_name=index.name
    )
    return vector_store

print("Document processing functions defined successfully!")

Document processing functions defined successfully!


In [12]:
# Create a sample document
sample_text = """# Company FAQ

## General Questions
Q: What are your business hours?
A: Our business hours are from 9:00 AM to 6:00 PM, Monday through Friday.

Q: How can I contact support?
A: You can reach our support team at support@example.com or call us at (555) 123-4567.

## Product Information
Q: What products do you offer?
A: We offer a range of software solutions including AI tools, data analytics platforms, and cloud services.

Q: Do you offer a free trial?
A: Yes, we offer a 14-day free trial for all our products.
"""

# Save the sample text to a file
with open("sample_faq.txt", "w") as f:
    f.write(sample_text)

print("Sample document created: sample_faq.txt")

Sample document created: sample_faq.txt


In [14]:
# First, install the required package
!pip install -q langchain-openai

# Then update the imports
from langchain_openai import OpenAIEmbeddings  # Updated import
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import os

def load_documents(file_path):
    """Load and split documents into chunks."""
    # Load document
    loader = TextLoader(file_path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    return text_splitter.split_documents(documents)

def create_vector_store(documents, index_name):
    """Create and return a vector store from documents."""
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

    # Create and store vectors in Pinecone
    vector_store = PineconeVectorStore.from_documents(
        documents=documents,
        embedding=embeddings,
        index_name=index_name
    )
    return vector_store

print("Document processing functions defined successfully!")

Document processing functions defined successfully!


In [16]:
# First, let's make sure we have all the right packages
!pip install -q langchain-openai pinecone-client

# Import required libraries
import os
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from pinecone import Pinecone

# Initialize Pinecone client
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "qa-bot-index"

def load_documents(file_path):
    """Load and split documents into chunks."""
    # Load document
    loader = TextLoader(file_path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    return text_splitter.split_documents(documents)

def create_vector_store(documents, index_name):
    """Create and return a vector store from documents."""
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

    # Create and store vectors in Pinecone
    vector_store = PineconeVectorStore.from_documents(
        documents=documents,
        embedding=embeddings,
        index_name=index_name
    )
    return vector_store

print("Document processing functions defined successfully!")

Document processing functions defined successfully!


In [3]:
# Install required packages
!pip install -q langchain-huggingface sentence-transformers faiss-cpu torch

# Import required libraries
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load a local embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': device}
)

# Load a local LLM with better configuration
model_name = "gpt2"  # Using a small model for demonstration
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Configure the text generation pipeline with better parameters
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,  # Increased from default
    temperature=0.7,     # Added for better response quality
    do_sample=True,      # Enable sampling for more diverse responses
    device=0 if device == "cuda" else -1
)

llm = HuggingFacePipeline(pipeline=pipe)

print("Models loaded successfully!")

def load_documents(file_path):
    """Load and split documents into chunks."""
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    return text_splitter.split_documents(documents)

def create_vector_store(documents):
    """Create and return a vector store from documents."""
    return FAISS.from_documents(documents, embeddings)

# Create a sample document
sample_text = """# Company FAQ

## General Questions
Q: What are your business hours?
A: Our business hours are from 9:00 AM to 6:00 PM, Monday through Friday.

Q: How can I contact support?
A: You can reach our support team at support@example.com or call us at (555) 123-4567.

## Product Information
Q: What products do you offer?
A: We offer a range of software solutions including AI tools, data analytics platforms, and cloud services.

Q: Do you offer a free trial?
A: Yes, we offer a 14-day free trial for all our products.
"""

# Save the sample text to a file
with open("sample_faq.txt", "w") as f:
    f.write(sample_text)

print("Sample document created: sample_faq.txt")

# Load and process the document
try:
    documents = load_documents("sample_faq.txt")
    print(f"Loaded {len(documents)} document chunks")

    # Create vector store
    vector_store = create_vector_store(documents)
    print("Vector store created successfully using FAISS!")

    # Create the QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True
    )

    print("QA system is ready! You can now ask questions about the document.")

    def ask_question(question):
        try:
            # Updated to use the new invoke method
            result = qa_chain.invoke({"query": question})
            print(f"\nQuestion: {question}")
            print(f"Answer: {result['result']}")

            if 'source_documents' in result and result['source_documents']:
                print("\nSources:")
                for i, doc in enumerate(result['source_documents']):
                    print(f"{i+1}. {doc.page_content[:100]}...")
            else:
                print("\nNo sources found for this answer.")

        except Exception as e:
            print(f"\nError processing question: {str(e)}")
            print("Trying with a simpler approach...")
            # Fallback to direct retrieval if the QA chain fails
            docs = vector_store.similarity_search(question, k=1)
            if docs:
                print(f"\nMost relevant information found:")
                print(docs[0].page_content)
            else:
                print("No relevant information found in the documents.")

    # Test the QA system
    questions = [
        "What are your business hours?",
        "Do you offer a free trial?",
        "How can I contact support?",
        "What products do you offer?"
    ]

    for question in questions:
        ask_question(question)
        print("\n" + "="*80 + "\n")

    # Interactive mode
    print("Enter 'quit' to exit the QA system.")
    while True:
        user_question = input("\nAsk a question about the document: ")
        if user_question.lower() == 'quit':
            break
        ask_question(user_question)

except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("\nTroubleshooting steps:")
    print("1. Check if you have enough disk space (at least 2GB free)")
    print("2. Make sure you have a stable internet connection")
    print("3. Try restarting the runtime (Runtime > Restart runtime) and running the code again")
    print("4. If the issue persists, try using a smaller model or different parameters")

Using device: cpu


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Models loaded successfully!
Sample document created: sample_faq.txt
Loaded 1 document chunks
Vector store created successfully using FAISS!
QA system is ready! You can now ask questions about the document.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Question: What are your business hours?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

# Company FAQ

## General Questions
Q: What are your business hours?
A: Our business hours are from 9:00 AM to 6:00 PM, Monday through Friday.

Q: How can I contact support?
A: You can reach our support team at support@example.com or call us at (555) 123-4567.

## Product Information
Q: What products do you offer?
A: We offer a range of software solutions including AI tools, data analytics platforms, and cloud services.

Q: Do you offer a free trial?
A: Yes, we offer a 14-day free trial for all our products.

Question: What are your business hours?
Helpful Answer:

Q: What are your business hours?

A: Our business hours are from 9:00 AM to 6:00 PM, Monday through Friday.

Q: Do you offer a free trial?

A: Yes, we offer a 14-day free trial for all our products.

Q: Do you offer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Question: Do you offer a free trial?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

# Company FAQ

## General Questions
Q: What are your business hours?
A: Our business hours are from 9:00 AM to 6:00 PM, Monday through Friday.

Q: How can I contact support?
A: You can reach our support team at support@example.com or call us at (555) 123-4567.

## Product Information
Q: What products do you offer?
A: We offer a range of software solutions including AI tools, data analytics platforms, and cloud services.

Q: Do you offer a free trial?
A: Yes, we offer a 14-day free trial for all our products.

Question: Do you offer a free trial?
Helpful Answer: We offer an unlimited number of free introductory trial accounts, free trial accounts for all our products, and free trial accounts for a variety of other products.

Q: Do you offer a free trial?

Helpful Answer: We offer 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Question: How can I contact support?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

# Company FAQ

## General Questions
Q: What are your business hours?
A: Our business hours are from 9:00 AM to 6:00 PM, Monday through Friday.

Q: How can I contact support?
A: You can reach our support team at support@example.com or call us at (555) 123-4567.

## Product Information
Q: What products do you offer?
A: We offer a range of software solutions including AI tools, data analytics platforms, and cloud services.

Q: Do you offer a free trial?
A: Yes, we offer a 14-day free trial for all our products.

Question: How can I contact support?
Helpful Answer:

Q: What's the best way to get the most out of your mobile phone?

A: We have developed a tool to help you find the best mobile phone solution for you. If you're interested in learning more about mobile phones, check out o